In [7]:
from  ipywidgets  import interact
import numpy as np
import pandas as pd
from datetime import date
import mibian

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("portfolio_records.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
eq_plot = figure(title="Portfolio Equity", plot_height=500, plot_width=900, y_range=(9800,18000), x_axis_type="datetime")
r = eq_plot.line(source=df, x='timestamp', y='portfolio_equity')
eq_plot.add_tools(HoverTool(tooltips=[
        ('date','@timestamp{%F}'),
        ('equity','@portfolio_equity')
    ],
    formatters={
        'timestamp':'datetime', 
        'equity':'printf',
        
    }
))

show(eq_plot, notebook_handle=True)

monthly_returns = []
returns_df = pd.DataFrame()
cur_month = 1
prev_equity = df.at[0,'portfolio_equity']

for index, row in df.iterrows():
    if index == (len(df.index) - 1) or df.at[index + 1,'timestamp'].month != cur_month:
        perc_change = (row['portfolio_equity'] - prev_equity) / prev_equity * 100    
        perc_change = float("{0:.2f}".format(perc_change))
        returns_df.at['2017', df.at[index,'timestamp'].strftime("%B")] = perc_change
        prev_equity = row['portfolio_equity']
        cur_month += 1
        
def color_negative_red(val):
    return 'background-color: #FBA38D' if val < 0 else 'background-color: #D0FB8D'  
returns_df = returns_df.style.applymap(color_negative_red)

returns_df

,January,February,March,April,May,June,July,August,September,October,November,December
2017,12.25,15.59,2.55,5.17,-1.47,3.72,0.36,6.07,2.28,3.28,1.66,0.33


In [12]:
class option(object):
   def  __init__(self, strike, price, type, units, action):
        self.strike = strike
        self.type = type
        self.price = price
        self.units = units
        self.action = action
        optDat = mibian.BS([61.77, self.strike, 1.45, 56], putPrice=(self.price/1000))
        self.impliedVolatility = optDat.impliedVolatility
        
p00 = option(56.5,750,'P', 1,1)
p01 = option(60,1695,'P', 2,-1)
p02 = option(63,3170,'P', 1,1)

options = [p00, p01, p02]

underlying  = [p/10 for p in range(450, 750)]
vals = []
theoVals = []

for val in underlying:
    tot_val = 0.0    
    for contract in options:            
        if val > contract.strike:
            tot_val += (0 -  contract.price) * contract.action * contract.units;
        else:
            tot_val += (((contract.strike - val)*1000) - contract.price) * contract.action * contract.units
    vals.append(tot_val)
    
for val in underlying:
    tot_val = 0.0    
    for contract in options:
        calc = mibian.BS([val, contract.strike, 1.45, 56], volatility = contract.impliedVolatility)
        tot_val += ((calc.putPrice * 1000) - contract.price) * contract.action * contract.units
        
    theoVals.append(tot_val)              

In [23]:
risk_plot = figure(title="Risk Graph", plot_height=500, plot_width=700)
r = risk_plot.line(underlying, vals, line_width=1.5)
r = risk_plot.line(underlying, theoVals, line_color="orange")
risk_plot.add_tools(HoverTool(tooltips=[
        ('underlying','@x'),
        ('pnl','@y')
    ],
    formatters={
        'underlying':'printf',
        
    }
))

show(risk_plot, notebook_handle=True)